# <center> **Web Scraping** </center>

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)

Nesta etapa será realizado a tecnica de web scraping, para coletar dados históricos de 2010 a 2020, registrados em formato tabular, com informações dos lançamentos do foguete Falcon 9, de uma página da Wikipedia intitulada ``List of Falcon 9 and Falcon Heavy launches``.

link:

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


Os dados a seram extraidos das tabelas, são os pertencentes as seguintes variaveis: ``Date and time(UTC)``, ``Version Booster``, ``Launch site``, ``Payload``, ``Payload mass``, ``Orbir``, ``Customer``, ``Launch outcome`` e ``Booster landing``

Abaixo segue uma amostra ilustrativa, de uma das tabelas a qual seram raspados os dados, e em seguida estrutados.

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)

### **Objetivos**

* Web scrap dos registros das tabelas HTML de lançamento do Falcon 9, que constam publicadas na Wikipedia.

* Analisar a tabela e converte la em um quadro de dados Pandas Dataframe.

In [255]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Formulando funções que iram auxiliar ao processar as tabelas HTML na raspagem de dados

In [256]:
def date_time(table_cells):
    """
    Esta função Extrai dos dados da coluna de data e tempo(Date and time) da tabela HTML
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

In [257]:
def booster_version(table_cells):
    """
    Esta função extrai os dados da coluna version booster da tabela HTML
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

In [258]:
def landing_status(table_cells):
    '''
    Esta função extrai os dados da coluna landing status da tabela HTML
    '''
    out = [i for i in table_cells.strings][0]
    return out

In [259]:
def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

In [260]:
def extract_column_from_header(row):
    '''
   Esta função extrai outros dados referente a coluna landing status da tabela HTML
    '''
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    column_name = ' '.join(row.contents)

    # Filtrar os dígitos e nomes vazios
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name

In [261]:
# Endereço da URL para requisitar conteudo HMTL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Objeto HTTP para solicitação de conteudo HTML
response = requests.get(static_url)
print(response.status_code,
      ' <--- 200 = OK')

# Objeto BeautifulSoup para aplicação em conteudos HTML
soup = BeautifulSoup(response.text,
                     'html.parser')

200  <--- 200 = OK


Imprima o título da página para verificar se o objeto BeautifulSoup foi criado corretamente

In [262]:
# Utilizar o comando .title
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### Capturar todos os nomes de colunas/variáveis ​​do cabeçalho da tabela HTML como exemplificado pela ilustração acima

In [263]:
# Obter os nomes de identificação das variaveis que constam no cabeçalho das tabelas
html_tables = soup.find_all('table')

# Selecionada a tabela para extrair os nomes
launch_table = html_tables[2]

Em seguida, se faz necessário apenas iterar pelos elementos ``<th>`` e aplicar a função **extract_column_from_header()** para extrair o nome das colunas uma por uma

In [264]:
# Armazena os nomes das colunas
columns = []

# Aplica o comando find_all() no elemento ``th``
headers = launch_table.find_all('th')

# Iterar cada elemento ``th`` a função ``extract_column_from_header()`` para obter os nomes
for header in headers:
    column = extract_column_from_header(header)

    # Evitar possivel erro a e captura de nulos/ausentes
    if column is not None and len(column) > 0:
        columns.append(column)

# Autenticando o exito do procediento
print(columns)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Criar uma nova estrutura de dados, com as tabelas HTML, um dicionário vazio,  como chave nesse dicionario, os nomes de colunas extraídos. Esse dicionário será convertido em um pandas dataframe

In [265]:
# Inicializando o dicionario com as chaves
launch_dict = dict.fromkeys(columns)

# Remover uma coluna irrelevante
del launch_dict['Date and time ( )']

# Vamos inicializar o launch_dict com cada valor para ser uma lista vazia
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Adicionadas algumas novas colunas
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Em seguida, é preciso apenas preencher o dicionario ``launch_dict`` com registros de lançamento extraídos de linhas de tabela HTML.

Normalmente, raspagem de dados em paginas HTML, podem provavelmente conter dados inesperadas e outros tipos de ruídos, insconsistencias e valores ausentes

In [266]:
extracted_row = 0

# Itera em cada tabela HTML existente na pagina
for table_number, table in enumerate(soup.find_all('table', 'wikitable plainrowheaders collapsible')):

  # Itera com cada linha da tabela HTML
  for rows in table.find_all('tr'):

        # Captura somente os valores que correspondem ao numero de voo ``Flight No.``
        if rows.th:
           if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        # Captura todos os elementos que correspondem a tag "td" da tabela HTML
        row=rows.find_all('td')

        ## Extrai os dados capturados referente a tag "td" e salvaas em um dicionário
        if flag:
            extracted_row += 1

            # Adiciona os dados de Flight No. ao dicionario launch_dict
            launch_dict['Flight No.'].append(flight_number)
            #print(flight_number)
            datatimelist=date_time(row[0])

            # Adiciona os dados de launch site ao dicionario launch_dict
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)

            # Adiciona os dados de payload ao dicionario launch_dict
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)

            # Adiciona os dados de payload mass ao dicionario launch_dict
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)

            # Adiciona os dados de Orbit ao dicionario launch_dict
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)

            # Adiciona os dados de Customer ao dicionario launch_dict
            if row[6].a is not None:
                customer = row[6].a.string
            else:
                customer = row[6].string.strip()
            launch_dict['Customer'].append(customer)
            #print(customer)

             # Adiciona os dados de Launch outcome ao dicionario launch_dict
            launch_outcome = list(row[7].strings)[0].replace('\n', '')
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)

            # Adiciona os dados de Data (date) ao dicionario launch_dict
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)

            # Adiciona os dados de tempo (time) no dicionario launch_dict
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)

            # Adiciona os dados de Booster Version ao dicionario launch_dict
            if row[1].a is not None:
                booster_version = row[1].a.string
            else:
                booster_version = row[1].string.strip()
            launch_dict['Version Booster'].append(booster_version)
            #print(bv)

            # Adiciona os dados de Booster landing ao dicionario launch_dict
            booster_landing = landing_status(row[8]).replace('\n', '')
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)


Após a extração e estruturação dos dados no dicionario, gerar por meio do dicionario um pandas dataframe

In [267]:
df_spacex = pd.DataFrame(
    {key:pd.Series(value) for key,
     value in launch_dict.items() })

In [268]:
print(df_spacex.shape)
df_spacex.head()

(121, 11)


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.0,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0,No attempt,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.0,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.0,No attempt,1 March 2013,15:10


### Salvando os dados estruturados em pandas dataframe, em um arquivo CSV

In [269]:
df_spacex.to_csv('Coletar_Dados_WebScrap_SpaceX.csv',
                 index=False,
                 index_label=False)